# Convex Optimization: Duality

**Prerequisites**

- Convex Optimization: Linear Programming

**Outcomes**

- Understand intuitions behind the dual and primal problem
- Understand the interpretation of a shadow price
- Know the link between slack variables and shadow prices

In [ ]:
import numpy as np
from scipy.optimize import linprog

## Review product mix

Let's consider a simplified product mix problem with just two products and two resources:

\begin{align*}
\max_x \ & 250 x_1 + 495 x_2 & \text{(profit)} \\
\mbox{subject to } \ & x_1 \le 5 & \text{(production capacity for } x_1  \text{)} \\
& x_1 \le 10 & \text{(production capacity for } x_2  \text{)} \\
& 0.5 x_1 + x_2 \le 6  & \text{(resource constraint 1)} \\
& 0.5 x_1 + 0.4 x_2 \le 8  & \text{(resource constraint 2)} \\
& x \ge 0 & \text{(non-negative production)}\\
\end{align*}


We know how to convert this to a linear program in general form:

$$\begin{align*}
\min_x \ & (-p)^T x \\
\mbox{subject to } \ & \begin{bmatrix} H \\ I \end{bmatrix}  x \le \begin{bmatrix} m \\ d \end{bmatrix} \\
& x \ge 0\\
\end{align*}$$

with $H$, $p$, $d$, $m$ defined as in the code cell below

In [ ]:
p = np.array([250., 495.])  # price vector
d = np.array([5., 10.,])    # production capacity
m = np.array([6., 8.])      # resource constraints

H = np.array([              # resource usage
    [0.5, 1],
    [0.5, 0.4]
])

We can solve with `scipy.optimize.linprog`

In [ ]:
c = -p
b = np.concatenate((m, d))
A = np.vstack((H, np.eye(len(p))))

sol_x = linprog(c=c, A_ub=A, b_ub=b)
sol_x

### Thought experiment

Suppose we don't know linear programming, but are asked to maximize profits

Because we don't know LP, we can't solve for optimal production decisions and profit

Also suppose that we have a "buyer" who is eager to purchase capacity from us

The buyer would be happy to purchase production capacity for goods $x_1$ and $x_2$ and/or the raw materials $m_1$ and $m_2$ themselves

To engage with the buyer, we need to settle on a price for each of our 4 resources

Call these prices

- $y_1$: resource 1
- $y_2$: resource 2
- $y_3$: production of $x_1$
- $y_4$: production of $x_2$


A starting point begin negotiating is that $y \ge 0$

We also acknowledge that we have a *technology* whereby we can convert raw materials and production capacity into final goods

In order for sale of our resources to entertain us, we must receive at least the price of the goods we could create using our technology

This leads to two other constraints:

\begin{align*}
0.5 y_1 + 0.5 y_2  + y_3 & \ge 250 \quad & \text{(technology for $x_1$)}\\
y_1 + 0.4 y_2  + y_4& \ge 495 \quad & \text{(technology for $x_2$)}
\\
\end{align*}

The first inequality states that selling the bundle used to power the technology used to produce one unit of good $x_1$ must provide us with *at least* the value of producing a unit of $x_1$ and selling at our price of 250

The second inequality is similar, but for good $x_2$

We give our buyer this information, as well as the full capacity we have on hand (the vector $b$)

The buyer does know linear programming and decides formulate the following problem in order to determine prices:

\begin{align*}
\min_y \; & 6 y_1 + 8 y_2 + 5 y_3 + 10 y_4  & \text{(cost of capacity)} \\
\text{s.t. } & y \ge 0 & \text{(non-negative prices)}\\
&0.5 y_1 + 0.5 y_2 + y_3  \ge 250 \quad & \text{(technology for $x_1$)}\\
&y_1 + 0.4 y_2 + y_4  \ge 495 \quad & \text{(technology for $x_2$)}
\end{align*}

The buyer solves this problem with `scipy.optimize.linprog`

In [ ]:
cy = np.array([6, 8, 5, 10])

# negative on A and b to flip the inequality
Ay = -np.array([[0.5, 0.5, 1, 0], [1, 0.4, 0, 1]])
by = -np.array([250, 495])

sol_y = linprog(cy, Ay, by)
sol_y

Let's view both the buyer's solution, and our solution

In [ ]:
sol_y

In [ ]:
sol_x

Notice that the cost to the buyer (`sol_y.fun`) is equal to the profit we get from running our factory (`-sol_x.fun`)!

This is *not* coincidence

Notice the relationship between the optimal prices (`sol_y.x`) and the slack values fromthe original problem (`sol_x.slack`):

- Whenever `sol_x.slack[i] == 0`, `sol_y.x[i] > 0`
- Also, when `sol_x.slack[i] > 0`, `sol_y.x[i] == 0`

This is also not coincidence... here's why

When slack variable `i` equals zero means that the constraint was *binding*, or that `(A @ x)[i] == b[i]`

We would like to be able have the constraint loosened, meaning an increase in `b[i]`

There is a price at which we break even for paying to loosening this constraint

This price is exactly `sol_y.x[i]`

On the other hand, for slack variable `j`, where `sol_x.slack[j] > = 0` the constraint is called *slack*

We would not be willing to pay anything in order to increase `b[j]`

For this reason `sol_y.x[j] == 0`

The variables $y$ are known as *shadow prices*

A shadow price captures the *value* of loosening a constraint

## The Dual

We originally solved a profit maximization profit for the factory operator

We then worked through what a potential buyer would offer us to buy out our production capacity

These two problems are intrinsically linked..

For any linear program (like the profit maximization one we started with) there is always a secondary linear program that solves for the shadow prices

The original linear program is called the *primal problem*, while the problem that finds shadow prices is called the *dual problem*

### General Dual

There is a formula we can follow to convert a the primal form of a general linear program into its dual

Suppose we start with

\begin{align*}
\min_x \quad & c^Tx \\
\text{s.t.}\quad & Ax \le b \\
& x \ge 0
\end{align*}

The dual is then

\begin{align*}
\min_y \quad & b^T y \\
\text{s.t.} \quad & (-A^T)y \le c \\
& y \ge 0
\end{align*}

We can verify this transformation for our example problem:

In [ ]:
cy - b

In [ ]:
by - c

In [ ]:
Ay - (-A.T)

**Exercise**

> Note: This is a modified version of an exercise from chapter 15 of "Labs for Foundations of Applied Mathematics Volume 2" by Jeffrey Humpherys And Tyler J. Jarvis released under the [Creative Commons Attribution 3.0 United States license](http://creativecommons.org/licenses/by/3.0/us/). The original source for the lab materials can be found at [https://github.com/Foundations-of-Applied-Mathematics/Labs](https://github.com/Foundations-of-Applied-Mathematics/Labs)

The DataFrame `foods` below has nutritional and price information for 18 foods commonly consumed by university students

In [ ]:
import pandas as pd
foods = pd.read_csv("foods.csv")
foods

**Part 1**
Your task is to formulate and solve linear program to solve for the cheapest possible diet, subject to some nutritional guidelines.

Each day you must consume

- No more than 2000 calories
- No more than 65 grams of fat
- No more than 50 grams of sugar
- At least 1000 milligrams of calcium
- At least 25 grams of fiber
- At least 46 grams of protein

*Hint 1*: use the `.to_numpy` method on a DataFrame or series to get the corresponding numpy array

*Hint 2*: the units for the nutrient columns of `foods` are consistent with what appears in the list above -- there is no need to change units

In [ ]:
def formulate_diet_primal(foods):
    """
    Given a DataFrame foods with the format from above,
    Construct `c`, `A` and `b` that can be passed to 
    `scipy.optimize.linprog` to solve the diet-budget problem
    described above
    
    Make sure to return them in the order c, A, b in order
    to pass tests below
    
    Also, please ensure that the rows of A and b are consistent
    with the order of the list above (calories, fat, sugar, calcium
    fiber, protein)
    """
    # construct c, A, and b
    # return them!
    pass

**Part 2**

Which constraints are binding?

How do you know?

Your answer here

**Part 3**

Formulate and solve the dual problem

How much would it cost to loosen the binding constraints?

*Bonus*: In what units would these costs need to be paid? Why?

In [ ]:
def formulate_diet_dual(foods):
    """
    Given a DataFrame `foods` with the format from above,
    construct the c, A, and b arrays needed to solve the 
    dual problem associated with the budget-minimization
    primal from before.    
    """
    # your code here
    pass

## Note

There is quite a bit of math going on behind the scenes

We opted for a "intuition motivated" explanation of how the primal and dual relate to one another, as well as the interpretation of the shadow prices and slack variables

For a more complete theoretical treatment of these concepts we refer you to the following resources:

- https://web.stanford.edu/~ashishg/msande111/notes/chapter4.pdf
- http://web.mit.edu/15.053/www/AMP-Chapter-04.pdf
- Chapter 5 of the Boyd textbook

During your studies, you will learn more about the following:

- Weak and strong duality theories
- Separating hyperplane theorem
- Farkas' Lemma
- Complementary slackness conditions
- The Karush Kuhn Tucker (KKT) conditions

We may return to some of the ideas in later lectures

If we do, we will cover necessary pre-requisites at that time